## Convolution based on... (DIMENSION TO DO CONVOLUTION ON) 
- Gene functionality (GO analysis) 
- Each convolutional layer is the top k genes correlated with the target genes (i.e. n_top = 5) 
- Random 

## Thungs to try instead of correlation
- Euclidean distance in n-dim array 
- Manhattan distance (look into it?) 
- Root Mean Squared Error 

## Activation functions to test (if I understand this correctly)
- ReLU (default) 
- Sigmoid (all positive, "gradual") 
- Softplus (gradual ReLU) 

## Loss function...
"The loss function to use during training is typically the mean squared error, but if you have a lot of outliers <br />
in the training set, you may prefer to use the mean absolute error instead. Alternatively, you can use the Huber <br />
loss, which is a combination of both."

## Dropout layer 
- "...closer to 20–30% in recurrent neural nets, and closer to 40–50% in convolutional neural networks."

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import scanpy as sc

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
import keras.losses

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Data Preparation

In [4]:
adata = sc.read_10x_mtx(
    '/Volumes/Samsung_T5/ResearchData/scanpyTutorial/data/filtered_gene_bc_matrices/hg19',  # the directory with the `.mtx` file
    var_names='gene_symbols',                      # use gene symbols for the variable names (variables-axis index)
    cache=True)

adata.var_names_make_unique()

In [5]:
data = pd.DataFrame.sparse.from_spmatrix(adata.X)
print('Working on {} cells and {} genes'.format(*data.shape))

Working on 2700 cells and 32738 genes


In [6]:
VMR = data.std() / data.mean()  # coefficient of variation: stdDev/mean (relative magnitude of std dev)
VMR[np.isinf(VMR)] = 0  # if mean is 0, we would get an infinite value --> set it to zero
potential_pred = data.columns[VMR > 0]

In [7]:
# Create covariance matrix (pd data frame),
covariance_matrix = pd.DataFrame(np.abs(np.corrcoef(data[potential_pred], rowvar= False)), # rowvar=False -> each column represents variable
    index=potential_pred,
    columns=potential_pred).fillna(0) # setting column names and fill NA with 0

covariance_matrix.shape; max(pd.Series.max(covariance_matrix)); min(pd.Series.min(covariance_matrix))


(16634, 16634)

1.0

0.0

In [7]:
# data_pred_unnorm = data.loc[:,potential_pred] # UNNORMALIZED

# gene_metric = (data_pred_unnorm.var()/(1+data_pred_unnorm.mean())).sort_values(ascending=False)
# gene_metric = gene_metric[gene_metric > 0]

# data_pred = np.log1p(data_pred_unnorm).astype(np.float32)

In [8]:
data_pred_full = data.loc[:,potential_pred] # UNNORMALIZED

gene_metric = (data_pred_full.var()/(1+data_pred_full.mean())).sort_values(ascending=False)
gene_metric = gene_metric[gene_metric > 0]

data_pred_full.shape

(2700, 16634)

In [9]:
# separating 10% that will be untouched during cross-validation
untouched_10_index = np.random.choice(data_pred_full.index, int(0.10 * data_pred_full.shape[0]), replace=False)
untouched_10 = data_pred_full.loc[untouched_10_index,:]
untouched_10.shape

touched_90_index = np.setdiff1d(data_pred_full.index, untouched_10_index)
data_pred = data_pred_full.loc[touched_90_index,:]
data_pred.shape

(270, 16634)

(2430, 16634)

In [10]:
# Defining target variable (y = 512 genes, x = 1800 ish genes)
batch = 1
indx = range((512*(batch-1)),(512*batch))
print('Working on Batch {}'.format(batch))

target_index = gene_metric.index[indx]
target = data_pred.loc[:,target_index]
target.shape

not_target_index = gene_metric.index[list(set(range(16634)) - set(indx))]
not_target = data_pred.loc[:,not_target_index]
not_target.shape

Working on Batch 1


(2430, 512)

(2430, 16122)

In [11]:
subMatrix = covariance_matrix.loc[target_index, not_target_index]
subMatrix.shape; max(pd.Series.max(subMatrix)); min(pd.Series.min(subMatrix))

(512, 16122)

0.9896605340015643

0.0

In [12]:
x = []
for i in range(512):
    indx = pd.Series.nlargest(subMatrix.iloc[i,:], 5).index
    x.extend(pd.Index.tolist(indx))
predictor_index = set(x) #keep unique only
predictor = data_pred.loc[:,predictor_index]
predictor.shape

(2430, 1425)

# Trying Cross Validation 

In [13]:
# Normal way
X_train, X_test, y_train, y_test = train_test_split(np.asarray(predictor), np.asarray(target), test_size=0.05)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2308, 1425) (2308, 512)
(122, 1425) (122, 512)


In [15]:
def wMSE(y_true, y_pred, binary=False):
    if binary:
        weights = tf.cast(y_true>0, tf.float32)
    else:
        weights = y_true
    return tf.reduce_mean(weights*tf.square(y_true-y_pred)) 


In [16]:
num_folds = 10
no_epochs = 200
batch_size = 64 
verbosity = 1

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
corr_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)


In [17]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    # Define the model architecture
    model = Sequential([ 
        Input(shape= inputs.shape[1]), 
        Dense(units=256, activation="relu"), 
        Dropout(rate=0.2),
        Dense(units=512, activation="softplus")
    ])

    # Compile the model
    model.compile(optimizer=Adam(lr=0.0001), 
                  loss=wMSE, 
                  metrics=["accuracy"])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Coverting to Numpy arrays
    inTrain = np.asarray(inputs[train]).astype(np.float32)
    outTrain = np.asarray(targets[train]).astype(np.float32)
    inTest = np.asarray(inputs[test]).astype(np.float32)
    outTest = np.asarray(targets[test]).astype(np.float32)

    # Fit data to model
    history = model.fit(inTrain, outTrain,
                        batch_size=batch_size,
                        epochs=no_epochs,
                        verbose=verbosity)

    # Generate generalization metrics
    scores = model.evaluate(inTest, outTest, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Calculating Correlation
    Y_test_raw = np.hstack(outTest).flatten()
    Y_test_imputed = np.hstack(model.predict(inTest)).flatten()

    Y_test_imputed = Y_test_imputed[Y_test_raw>0]
    Y_test_raw = Y_test_raw[Y_test_raw>0]

    corrFold = pearsonr(Y_test_raw,Y_test_imputed)[0],
    print(f'Correlation: {corrFold[0]}')
    corr_per_fold.append(corrFold)

    # Increase fold number
    fold_no = fold_no + 1


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/200
35/35 [==============================] - 0s 3ms/step - loss: 4842.6006 - accuracy: 0.1139
Epoch 2/200
35/35 [==============================] - 0s 3ms/step - loss: 4757.3506 - accuracy: 0.3937
Epoch 3/200
35/35 [==============================] - 0s 3ms/step - loss: 4576.0132 - accuracy: 0.4390
Epoch 4/200
35/35 [==============================] - 0s 3ms/step - loss: 4265.6401 - accuracy: 0.4435
Epoch 5/200
35/35 [==============================] - 0s 3ms/step - loss: 3833.2485 - accuracy: 0.4431
Epoch 6/200
35/35 [==============================] - 0s 3ms/step - loss: 3355.4756 - accuracy: 0.4417
Epoch 7/200
35/35 [==============================] - 0s 3ms/step - loss: 2882.6482 - accuracy: 0.4422
Epoch 8/200
35/35 [==============================] - 0s 3ms/step - loss: 2476.1562 - accuracy: 0.4422
Epoch 9/200
35/35 [==============================] - 0s 3ms/step - loss: 2152.3811 - ac

In [18]:
# == Provide average scores ==
print('-----------------------------------------------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('--------------------------------------------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}% - Correlation: {corr_per_fold[i]}')
print('-----------------------------------------------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Correlation: {np.mean(corr_per_fold)}')
print('-----------------------------------------------------------------------------------------------------------------')

-----------------------------------------------------------------------------------------------------------------
Score per fold
--------------------------------------------------------------------------------------------------------------
> Fold 1 - Loss: 1385.6485595703125 - Accuracy: 54.7325074672699% - Correlation: (0.7820328146518938,)
--------------------------------------------------------------------------------------------------------------
> Fold 2 - Loss: 735.9766235351562 - Accuracy: 58.847737312316895% - Correlation: (0.8149358563379714,)
--------------------------------------------------------------------------------------------------------------
> Fold 3 - Loss: 1311.497802734375 - Accuracy: 61.72839403152466% - Correlation: (0.8188446746393654,)
--------------------------------------------------------------------------------------------------------------
> Fold 4 - Loss: 966.0230712890625 - Accuracy: 56.37860298156738% - Correlation: (0.8123591951515028,)
--------------